# Clasificación con búsqueda automática de arquitectura de la red

In [1]:
import pandas as pd

En este segundo taller del curso trabajaremos con un dataset de datos de estudiantes de un curso de programación. Este dataset está compuesto de 53 variables binarias, que indican si el estudiante realizó o no uno de los 53 ejercicios disponibles en una plataforma de programación en línea. Cada estudiante tiene un estado que indica si aprobó o reprobó el curso al final del semestre. El objetivo es construir un clasificador que permita predecir si un estudiante aprobará o reprobará el curso de acuerdo a su actividad.

In [2]:
# Cargamos los datos
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
    name=fn
data = pd.read_csv(name, sep=";")

Saving dataset.csv to dataset.csv


In [3]:
data=pd.read_csv("dataset.csv",delimiter=";")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 54 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   estado  467 non-null    object
 1   e0      467 non-null    int64 
 2   e1      467 non-null    int64 
 3   e2      467 non-null    int64 
 4   e3      467 non-null    int64 
 5   e4      467 non-null    int64 
 6   e5      467 non-null    int64 
 7   e6      467 non-null    int64 
 8   e7      467 non-null    int64 
 9   e8      467 non-null    int64 
 10  e9      467 non-null    int64 
 11  e10     467 non-null    int64 
 12  e11     467 non-null    int64 
 13  e12     467 non-null    int64 
 14  e13     467 non-null    int64 
 15  e14     467 non-null    int64 
 16  e15     467 non-null    int64 
 17  e16     467 non-null    int64 
 18  e17     467 non-null    int64 
 19  e18     467 non-null    int64 
 20  e19     467 non-null    int64 
 21  e20     467 non-null    int64 
 22  e21     467 non-null    in

In [5]:
Y=data["estado"].astype("category").cat.codes

Podemos ver que hay un desbalance de datos, solamente 49 de los 467 estudiantes reprobaron el curso el semestre en donde se capturó la información. Esta situación es común en las tareas de clasificación, normalmente hay muchos menos casos de la clase que deseamos detectar. Situaciones así suceden también en ciberseguridad, en detección de enfermedades y en detección de fraudes.

In [6]:
data["estado"].value_counts()

,count
estado,
A,418
R,49


Tomaremos como base para el análisis solamente los primeros 4 ejercicios. Quedará como ejercicio para ti construir algunos modelos adicionales, seleccionando las variables más apropiadas para la predicción.

In [7]:
X=data[["e0","e1","e2","e3"]]

Como en otras ocasiones dividimos los datos en conjuntos de entrenamiento y test, pero en esta ocasión incluimos el parámetro stratify, que permite que las proporciones de las clases se mantengan entre los conjuntos de entrenamiento y test, eso nos asegura que hayan casos de todas las clases en todos los conjuntos (si hacemos una partición puramente aleatoria, podría uno de los conjuntos quedan sin representantes de alguna de las clases presentes en el problema)

In [8]:
from sklearn.model_selection import train_test_split
import numpy as np
X=X.to_numpy()
Y=Y.to_numpy()
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=3)

Hay varias formas de manejar el desbalance de clases, pero se pueden clasificar en dos tipos principales:

Mecanismos de Over Sampling, que aumentan los casos de la clase minoritaria para equiparar la proporción con la mayoritaria. Dentro de este grupo tenemos el oversampling puramente aleatorio, y el SMOTE, que crea nuevos casos usando interpolación de los valores presentes en el dataset. Esta técnica es muy útil cuando tenemos variables reales.

Mecanismos de Under Sampling, que disminuyen los casos de la clase mayoritaria, para equiparar las clases. Dentro de este grupo tenemos el random under sampling, que hace la disminución en forma completamente aleatoria y los Tomek Links. Esta última estrategia lo que hace es eliminar casos que están entre las fronteras entre las clases, asumiendo que su clasificación puede ser ambigua y confundir al clasificador.

Estas dos estrategias se pueden combinar también, por ejemplo con la estrategia SMOTETomek.

In [9]:
from imblearn.over_sampling import RandomOverSampler,SMOTE
from imblearn.under_sampling import RandomUnderSampler,TomekLinks
from imblearn.combine import SMOTETomek

Para nuestro ejemplo usaremos SMOTETomek, pero como ejercicio prueba como influye en el rendimiento del clasificador usar las otras estrategias de balanceo antes del entrenamiento.

In [10]:
oversampler=SMOTETomek(random_state=32)

El balanceo de clases solamente debe hacerse en el conjunto de entrenamiento, nunca en el conjunto de test, porque si balanceamos el conjunto de test estaríamos alterando las proporciones reales de los datos. Balancear el conjunto de entrenamiento no produce problemas, porque lo que permite es que el modelo pueda reconocer mejor los patrones que diferencian a las clases.

In [11]:
x_train_b,y_train_b=oversampler.fit_resample(x_train,y_train)

In [12]:
!pip install -q keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.7 MB/s eta 0:00:00


In [13]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Input

# ✅ importación correcta de Keras Tuner
from keras_tuner import HyperParameters, RandomSearch, Objective


In [14]:
from keras_tuner import RandomSearch, HyperParameters

def test_model(hp):
    model = Sequential()
    model.add(Dense(16, activation="relu", input_shape=(10,)))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

tuner = RandomSearch(
    test_model,
    objective="val_accuracy",
    max_trials=2,
    executions_per_trial=1,
    directory="test_tuner",
    project_name="demo"
)

print("✅ Keras Tuner funcionando correctamente")


✅ Keras Tuner funcionando correctamente


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Input
from keras.models import Sequential
import keras_tuner
from keras_tuner.engine.hyperparameters import Choice

Dado que hay múltiples arquitecturas posibles que se pueden usar para crear un clasificador, es necesario usar un método de búsqueda sistemático de la mejor combinación de capas y neuronas para la red. Esto se puede automatizar usando el modulo keras_tuner. Para hacerlo, solamente debemos definir una función que construyas la red a partir de un conjunto de valores posibles para los parámetros.

In [16]:
def build_model(hp):
    modelo=Sequential()
    modelo.add(Input(shape=(4,)))
    for i in range(hp.Choice("capas",[1,2,3])):
        modelo.add(Dense(hp.Choice("neuronas",[8,16,32,64])))
    modelo.add(Dense(1,activation="sigmoid"))
    modelo.compile(loss="binary_crossentropy",metrics=[tf.keras.metrics.BinaryAccuracy(name="acc")])
    return modelo

Una vez creada la función de creación de las redes candidatas, definimos la función de búsqueda, indicando la métrica que usaremos para guiar el proceso ("val_acc" accuracy en el conjunto de validación, en este caso), el número de pruebas por configuración (2) y las carpetas en donde almacenaremos los resultados.

In [17]:
tuner=keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective=keras_tuner.Objective("val_acc",direction="max"),
    max_trials=32,
    executions_per_trial=2,
    overwrite=False,
    directory="modelos",
    project_name="reprobacion"
)

Con esta función podemos ver el tamaño del espacio de búsqueda (la cantidad de combinaciones posibles de los parámetros que definimos)

In [18]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
capas (Choice)
{'default': 1, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
neuronas (Choice)
{'default': 8, 'conditions': [], 'values': [8, 16, 32, 64], 'ordered': True}


Ahora iniciamos la búsqueda, dando 8 iteraciones para el entrenamiento de cada configuración. Cuando determinemos la mejor red, podemos afinar su entrenamiento si es que en 8 iteraciones no hubiese convergido completamente.

In [19]:
tuner.search(x_train_b,y_train_b,epochs=8,validation_data=(x_test,y_test))

Trial 12 Complete [00h 00m 07s]
val_acc: 0.9255319237709045

Best val_acc So Far: 0.9255319237709045
Total elapsed time: 00h 01m 39s


En la primera posición del arreglo de mejores modelos, está aquel que obtuvo los mejores resultados.

In [20]:
mejor_modelo=tuner.get_best_models()[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Con Summary podemos ver su arquitectura.

In [21]:
mejor_modelo.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,545 (17.75 KB)

 Trainable params: 4,545 (17.75 KB)

 Non-trainable params: 0 (0.00 B)

Una vez obtenido el modelo, podemos grabarlo para analizarlo posteriormente.

In [22]:
mejor_modelo.save("mejor_modelo.keras")